In [ ]:
#IMPORTING LIBRARIES
import random
import threading
import time
import xml.etree.ElementTree as ET

# Define the ITstudent class
class ITstudent:
    def __init__(self, name, student_id, program, courses, marks):
        self.name = name
        self.student_id = student_id
        self.program = program
        self.courses = courses
        self.marks = marks
         
    #function for calculating average
    def calculate_average(self):
        total_marks = sum(self.marks)
        average = total_marks / len(self.marks)
        return average
    
    #function for displaying final result
    def is_passed(self):
        average = self.calculate_average()
        if average >= 50:
            return "Pass"
        else:
            return "Fail"

# Define the Buffer class
class Buffer:
    def __init__(self, max_size):
        self.max_size = max_size
        self.buffer = []
        self.semaphore = threading.Semaphore(1)
        self.empty = threading.Semaphore(self.max_size)
        self.full = threading.Semaphore(0)

    def insert(self, item):
        self.empty.acquire()
        self.semaphore.acquire()
        self.buffer.append(item)
        self.semaphore.release()
        self.full.release()

    def remove(self):
        self.full.acquire()
        self.semaphore.acquire()
        item = self.buffer.pop(0)
        self.semaphore.release()
        self.empty.release()
        return item
    

# Define the Producer class
class Producer(threading.Thread):
    def __init__(self, buffer):
        threading.Thread.__init__(self)
        self.buffer = buffer

    def run(self):
        for i in range(1, 11):
            student = generate_student(self)
            xml_file = create_xml_file(student, i)
            self.buffer.insert(i)
            time.sleep(random.randint(1, 3))
            
       #Function for generating student names
    def generate_name(self):
        names = random.choice(['Sebenele Dlamini','Lewis Makhafula', 'Senzangakhona Langa', 'Shaka Zulu','Jim Kane'])
        return names
    
    #Function for generating coarses
    def generate_courses(self):
        courses = ["Data Mining", "Algorithnms", "Innovation", "Machine learning","Operating Systems"]
        return random.sample(courses, random.randint(5,5))


# Function to generate a random student
def generate_student(self):
    name = self.generate_name()
    student_id = str(random.randint(20200001, 20209999))
    program = "Information Technology"
    courses = self.generate_courses()
    marks = [random.randint(0, 100) for _ in range(len(courses))]
    return ITstudent(name, student_id, program, courses, marks)

# Function to create an XML file for a student
def create_xml_file(student, index):
    root = ET.Element("Student")
    name = ET.SubElement(root, "Name")
    name.text = student.name
    student_id = ET.SubElement(root, "StudentID")
    student_id.text = student.student_id
    program = ET.SubElement(root, "Program")
    program.text = student.program
    courses = ET.SubElement(root, "Courses")
    for i in range(len(student.courses)):
        course = ET.SubElement(courses, "Course")
        course.text = student.courses[i]
        mark = ET.SubElement(course, "Mark")
        mark.text = str(student.marks[i])
    tree = ET.ElementTree(root)
    xml_file = f"student{index}.xml"
    tree.write(xml_file)
    return xml_file

# Define the Consumer class
class Consumer(threading.Thread):
    def __init__(self, buffer):
        threading.Thread.__init__(self)
        self.buffer = buffer

    def run(self):
        while True:
            item = self.buffer.remove()
            xml_file = f"student{item}.xml"
            student = parse_xml_file(xml_file)
            average = student.calculate_average()
            pass_fail = student.is_passed()
            print_student_info(student, average, pass_fail)
            time.sleep(random.randint(1, 3))



# Function to parse an XML file and create an ITstudent object
def parse_xml_file(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    name = root.find("Name").text
    student_id = root.find("StudentID").text
    program = root.find("Program").text
    courses = []
    marks = []
    for course in root.find("Courses"):
        courses.append(course.text)
        marks.append(int(course.find("Mark").text))
    return ITstudent(name, student_id, program, courses, marks)

# Function to print student information
def print_student_info(student, average, pass_fail):
    print("========================================================================================\n")
    print("Student Name:  ", student.name)
    print("Student ID:    ", student.student_id)
    print("Program:       ", student.program)
    print("Courses and Marks:")
    print("=================")
    for i in range(len(student.courses)):
        print("      ", student.courses[i], ":", student.marks[i])
    print("AVERAGE MARK:", average)
    print("FINAL RESULT ")
    print("Pass/Fail:", pass_fail)
    
    print("\n\n\n")

# Create the buffer
buffer = Buffer(10)

# Create the producer and consumer threads
producer = Producer(buffer)
consumer = Consumer(buffer)

# Start the threads
producer.start()
consumer.start()

# Wait for the threads to finish
producer.join()
consumer.join()



Student Name:   Senzangakhona Langa
Student ID:     20209174
Program:        Information Technology
Courses and Marks:
       Innovation : 70
       Algorithnms : 17
       Data Mining : 41
       Machine learning : 40
       Operating Systems : 74
AVERAGE MARK: 48.4
FINAL RESULT 
Pass/Fail: Fail





Student Name:   Senzangakhona Langa
Student ID:     20209636
Program:        Information Technology
Courses and Marks:
       Operating Systems : 75
       Machine learning : 1
       Innovation : 16
       Algorithnms : 25
       Data Mining : 13
AVERAGE MARK: 26.0
FINAL RESULT 
Pass/Fail: Fail





Student Name:   Jim Kane
Student ID:     20200170
Program:        Information Technology
Courses and Marks:
       Data Mining : 73
       Algorithnms : 66
       Operating Systems : 92
       Innovation : 21
       Machine learning : 47
AVERAGE MARK: 59.8
FINAL RESULT 
Pass/Fail: Pass





Student Name:   Jim Kane
Student ID:     20209645
Program:        Information Technology
Courses and M